#### Primero importamos las librerias necesarias

In [1]:
# Para convertirlo en un df
import pandas as pd
# Para cálculos numericos
import numpy as np
# Para graficar
import matplotlib.pyplot as plt
# Para normalizar los datos
from sklearn.preprocessing import MinMaxScaler

## ***1.- Recopilación de Datos***

In [2]:
df = pd.read_csv('datasets/titanik_train.csv', index_col=0)

In [3]:
# por si acaso creamos una copia del df
df_copy = df.copy()

## **2.- Exploración y Análisis de Datos**

In [4]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


##### Según donde saque este dataset, survived representa con un 1 si sobrevivieron y 0 si no

In [5]:
#vemos algunas metricas como la mediana, minimo, maximo etc..
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [7]:
df.shape

(891, 11)

In [8]:
df['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [9]:
# Vamos a eliminar la categoria de nombre y cabin, ya que no parece muy relevante para el modelo
columna_a_eliminar = 'Name'
df = df.drop(columna_a_eliminar, axis=1)

In [10]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,female,35.0,1,0,113803,53.1000,C123,S
5,0,3,male,35.0,0,0,373450,8.0500,NaN,S


## 3.- División de Datos

#### Hay que dividir los datos en entrenamiento, evaluación y prueba

In [11]:
# importamos train_test_split para dividir los datos
from sklearn.model_selection import train_test_split

train_set, val_y_pr_set = train_test_split(df, test_size=0.3, random_state=42)
val_set, test_set = train_test_split(val_y_pr_set, test_size=0.5, random_state=42)

In [12]:
print(f'la longitud de train_set es de: {len(train_set)}')
print(f'la longitud de val_set es de: {len(val_set)}')
print(f'la longitud de test_set es de: {len(test_set)}')

la longitud de train_set es de: 623
la longitud de val_set es de: 134
la longitud de test_set es de: 134


#### Ahora separamos las etiquetas (salidas 'y'), de las caracteristicas (entradas 'x')

In [13]:
# Primero el df original
x_df = df.drop('Survived', axis=1)
y_df = df['Survived'].copy()

In [14]:
# Ahora en el de entrenamiento 
x_train = train_set.drop('Survived', axis=1)
y_train = train_set['Survived'].copy()

In [15]:
# Ahora los de validación
x_val = val_set.drop('Survived', axis=1)
y_val = val_set['Survived'].copy()

In [16]:
# Ahora los de validación
x_test = test_set.drop('Survived', axis=1)
y_test = test_set['Survived'].copy()

In [17]:
x_val.shape

(134, 9)

In [18]:
y_val.shape

(134,)

## 4.- Preprocesamiento de datos

#### Antes de entrenar al módelo, hay que preparar los datos, como imputar, normalizar o estandarizar y codificar


In [19]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
446,1,male,4.0,0,2,33638,81.8583,A34,S
651,3,male,NaN,0,0,349221,7.8958,NaN,S
173,3,female,1.0,1,1,347742,11.1333,NaN,S
451,2,male,36.0,1,2,C.A. 34651,27.7500,NaN,S
315,2,male,43.0,1,1,F.C.C. 13529,26.2500,NaN,S


##### Los NaN son los valores nulos que hay que eliminar

In [20]:
columns_with_nulls = x_train.columns[x_train.isnull().any()]
print(columns_with_nulls)

Index(['Age', 'Cabin', 'Embarked'], dtype='object')


#### Estas son las columnas con valores nulos

### 4.1.- Imputar

In [21]:
#para imputar los datos importamos simpleimputer
from sklearn.impute import SimpleImputer

# seleccionamos las columnas numericas
numeric_columns = x_train.select_dtypes(include=['number']).columns

# seleccionamos las categorias
cat_columns = x_train.select_dtypes(include=['object']).columns

# Imputamos valores numéricos con la mediana
numeric_imputer = SimpleImputer(strategy='median')# instanciamos ese imputador numerico
x_train[numeric_columns] = numeric_imputer.fit_transform(x_train[numeric_columns])

# Imputamos las categorias con el valor mas frecuente
imputer_categorical = SimpleImputer(strategy='most_frequent')# instanciamos ese imputador categorico
x_train[cat_columns] = imputer_categorical.fit_transform(x_train[cat_columns])


In [22]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
446,1.0,male,4.0,0.0,2.0,33638,81.8583,A34,S
651,3.0,male,28.0,0.0,0.0,349221,7.8958,C23 C25 C27,S
173,3.0,female,1.0,1.0,1.0,347742,11.1333,C23 C25 C27,S
451,2.0,male,36.0,1.0,2.0,C.A. 34651,27.7500,C23 C25 C27,S
315,2.0,male,43.0,1.0,1.0,F.C.C. 13529,26.2500,C23 C25 C27,S


#### Los valores nulos se han eliminado, en cambio esta la media y la categoria mas frecuente

In [23]:
columns_with_nulls = x_train.columns[x_train.isnull().any()]
print(columns_with_nulls)

Index([], dtype='object')


In [24]:
x_train.shape

(623, 9)

### 4.2.- Normalizar

#### Ahora vamos a normalizar los datos, es decir, vamos a hacer que los valores no tengan mucha diferencia entre si, sino que tengan una escala entre 0 y 1

In [25]:
# lo haremos con minmaxscaler

# Creamos un objeto MinMaxScaler
scaler = MinMaxScaler()

# Seleccionamos solo las columnas numéricas para la normalización
numeric_cols = x_train.select_dtypes(include='number').columns

# Normalizar las columnas numéricas
x_train[numeric_cols] = scaler.fit_transform(x_train[numeric_cols])

# Ahora, el df contiene las columnas numéricas normalizadas
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
446,0.0,male,0.044986,0.000,0.333333,33638,0.159777,A34,S
651,1.0,male,0.346569,0.000,0.000000,349221,0.015412,C23 C25 C27,S
173,1.0,female,0.007288,0.125,0.166667,347742,0.021731,C23 C25 C27,S
451,0.5,male,0.447097,0.125,0.333333,C.A. 34651,0.054164,C23 C25 C27,S
315,0.5,male,0.535059,0.125,0.166667,F.C.C. 13529,0.051237,C23 C25 C27,S


In [26]:
x_train.shape

(623, 9)

### 4.3.- Codificar

#### El ultimo paso seria codificarlo, es decir, convertir las categorias (letras), en números

In [27]:
# Seleccionar columnas categóricas
cat_columns1 = x_train.select_dtypes(include=['object']).columns

# Aplicar One-Hot Encoding al conjunto de entrenamiento
x_train = pd.get_dummies(x_train, columns=cat_columns1, dtype=float)

In [28]:
x_train.shape

(623, 611)

In [29]:
x_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Ticket_110152,Ticket_110413,Ticket_110465,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
446,0.0,0.044986,0.000,0.333333,0.159777,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
651,1.0,0.346569,0.000,0.000000,0.015412,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
173,1.0,0.007288,0.125,0.166667,0.021731,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
451,0.5,0.447097,0.125,0.333333,0.054164,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
315,0.5,0.535059,0.125,0.166667,0.051237,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### ok, x_train ya esta, ahora hay que hacer lo mismo con x_val

In [30]:
x_val.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
478,3,male,29.0,1,0,3460,7.0458,NaN,S
378,1,male,27.0,0,2,113503,211.5000,C82,C
536,2,female,7.0,0,2,F.C.C. 13529,26.2500,NaN,S
307,1,female,NaN,0,0,17421,110.8833,NaN,C
31,1,male,40.0,0,0,PC 17601,27.7208,NaN,C


#### Lo se, es un coñaso empezar de nuevo, pues por eso estan los pipelines que lo aprenderemos en otro momento

In [31]:
#para imputar los datos importamos simpleimputer
from sklearn.impute import SimpleImputer

# seleccionamos las columnas numericas
numeric_columns = x_val.select_dtypes(include=['number']).columns

# seleccionamos las categorias
cat_columns = x_val.select_dtypes(include=['object']).columns

# Imputamos valores numéricos con la mediana
numeric_imputer = SimpleImputer(strategy='median')# instanciamos ese imputador numerico
x_val[numeric_columns] = numeric_imputer.fit_transform(x_val[numeric_columns])

# Imputamos las categorias con el valor mas frecuente
imputer_categorical = SimpleImputer(strategy='most_frequent')# instanciamos ese imputador categorico
x_val[cat_columns] = imputer_categorical.fit_transform(x_val[cat_columns])

In [32]:
# lo haremos con minmaxscaler

# Creamos un objeto MinMaxScaler
scaler = MinMaxScaler()

# Seleccionamos solo las columnas numéricas para la normalización
numeric_cols = x_val.select_dtypes(include='number').columns

# Normalizar las columnas numéricas
x_val[numeric_cols] = scaler.fit_transform(x_val[numeric_cols])

# Ahora, el df contiene las columnas numéricas normalizadas

In [33]:
# Seleccionar columnas categóricas
cat_columns1 = x_val.select_dtypes(include=['object']).columns

# Aplicar One-Hot Encoding al conjunto de entrenamiento
x_val = pd.get_dummies(x_val, columns=cat_columns1, dtype=float)

In [34]:
# Asegurarte de que las columnas en x_val coincidan con las columnas en x_train
missing_cols = set(x_train.columns) - set(x_val.columns)
for col in missing_cols:
    x_val[col] = 0  # Añadir columnas faltantes con valores de cero

# Asegurarte de que el orden de las columnas sea el mismo en ambos conjuntos
x_val = x_val[x_train.columns]


C:\Users\jhony\AppData\Local\Temp\ipykernel_17156\2925394198.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_val[col] = 0  # Añadir columnas faltantes con valores de cero
C:\Users\jhony\AppData\Local\Temp\ipykernel_17156\2925394198.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_val[col] = 0  # Añadir columnas faltantes con valores de cero
C:\Users\jhony\AppData\Local\Temp\ipykernel_17156\2925394198.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

In [35]:
x_val.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Ticket_110152,Ticket_110413,Ticket_110465,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
478,1.0,0.438990,0.25,0.0,0.026854,0.0,1.0,0,0,0,...,0,0.0,0.0,0,0,0,0,0.0,0.0,1.0
378,0.0,0.407823,0.00,0.5,0.806098,0.0,1.0,0,0,0,...,0,0.0,0.0,0,0,0,0,1.0,0.0,0.0
536,0.5,0.096151,0.00,0.5,0.100048,1.0,0.0,0,0,0,...,0,0.0,0.0,0,0,0,0,0.0,0.0,1.0
307,0.0,0.415615,0.00,0.0,0.422614,1.0,0.0,0,0,0,...,0,0.0,0.0,0,0,0,0,1.0,0.0,0.0
31,0.0,0.610410,0.00,0.0,0.105653,0.0,1.0,0,0,0,...,0,0.0,0.0,0,0,0,0,1.0,0.0,0.0


## 5.- Entrenamiento

In [36]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=5000)
model.fit(x_train, y_train)

LogisticRegression(max_iter=5000)

##### Ya tenemos el modelo 🎉
##### Ahora solo nos toca predecir nuevos valores y ver quetal se comporta este modelo

In [37]:
y_val_predic = model.predict(x_val)
# Estas son las 5 primeras predicciones de mi modelo para los datos x_val


## 6.- Evaluación

In [38]:
print(len(y_val))
print(len(x_val))

134
134


In [39]:
y_val.shape

(134,)

In [40]:
#from sklearn.metrics import confusion_matrix

#confusion_matrix(y_val, y_val_predict)